In [1]:
#functions here
def euclid_gcd(a, b):
    if a == 0:
        x, y = (0, 1)
        return (b, x, y)
    d, x1, y1 = euclid_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return (d, x, y)

def inverse(a, n):
    if (a < 0):
        a += n
    d, x, y = euclid_gcd(a, n)
    if (d == 1):
        return x

def solve_equation(a, b, n):
    x = list()
    d = euclid_gcd(a, n)[0] % n
    
    if d == 1: 
        a_inv = inverse(a, n)
        x.append(a_inv * b % n) 
        return x
    
    if b % d == 0:
        a1 = a // d
        b1 = b // d
        n1 = n // d
        x0 = solve_equation(a1, b1, n1)[0]
        for i in range(0, d):
            x.append(x0 + i * n1)
        return x
    return -1

In [2]:
with open('Var14.txt', 'rb') as f:
    text = f.read().decode('utf-8').strip()
text = text.replace('\n', '')

log = open('Output.txt', 'w')

In [3]:
alphabet = 'абвгдежзийклмнопрстуфхцчшщьыэюя'

In [4]:
bigr_count = 0

In [5]:
bigr_dict = dict()

In [6]:
for x in alphabet:
    for y in alphabet:
        key = x + y
        bigr_dict.update({key : 0})

In [7]:
temp = text
text_present = True

In [8]:
while text_present:
    bigramm = temp[:2]
    temp = temp[1:]
    
    if len(bigramm) < 2:
        text_present = False
    else:
        bigr_count += 1
        bigr_dict[bigramm] += 1

In [9]:
for key, val in bigr_dict.items():
    if val != 0:
        freq = val / bigr_count
        log.write('{} | {:6d} | {:0.5f}\n'.format(key, val, freq))

In [10]:
dict_top = dict(sorted(bigr_dict.items(),
                    key = lambda item: item[1],
                    reverse=True)[:5])

In [11]:
def get_bigr_num(bigram, alphabet):
    x1 = alphabet.index(bigram[0])
    x2 = alphabet.index(bigram[1])
    return len(alphabet) * x1 + x2

def get_bigr_text(num, alphabet):
    x1 = num // len(alphabet)
    x2 = num % len(alphabet)
    return alphabet[x1] + alphabet[x2]

def decrypt_text(A, B, text, alphabet):
    m = len(alphabet)
    decrypted = ''
    
    temp = text
    text_present = True
    while text_present:
        bigramm = temp[:2]
        temp = temp[2:]
        if len(bigramm) < 2:
            text_present = False
        else:
            Y = get_bigr_num(bigramm, alphabet)
            A_inv = inverse(A, pow(m, 2))
            X = (A_inv * (Y - B)) % pow(m, 2)
            
            decrypted += get_bigr_text(X, alphabet)
         
    return decrypted

def detect_lang(A, B, decrypted, alphabet):
    freq_dict = { x : 0 for x in alphabet }
    for char in decrypted:
        freq_dict[char] += 1
    
    top_lang = ['о', 'а', 'е', 'и', 'н', 'т']
    top_text = list(dict(sorted(freq_dict.items(),
                    key = lambda item: item[1],
                    reverse=True)[:6]).keys())
    top_chars = ''.join(x for x in top_text)
    if all( x in top_text for x in top_lang ):
        log.write('\nDecrypted text with A:{:3d} B:{:3d}. Top chars matching: {}.\n\n'.format(A, B, top_chars))
        log.write(decrypted)
        log.write('\n\n')
    else:
        log.write('Rejected text candidate A:{:3d} B:{:3d}. Top chars are: {}.\n'.format(A, B, top_chars))
    

In [12]:
top_text = list(dict_top.keys())
top_text_str = ''.join(x + ' ' for x in top_text)
log.write('Top bigramms in text are: {} \n'.format(top_text_str))

43

In [13]:
top_lang = ['ст', 'но', 'то', 'на', 'ен']

In [14]:
for clear1 in range(0, 5):
    for encr1 in range(0, 5):
        for clear2 in range(0, 5):
            for encr2 in range(0, 5):
                if clear1 == clear2 or encr1 == encr2:
                    continue
                else:
                    lang1 = get_bigr_num(top_lang[clear1], alphabet) # X*
                    text1 = get_bigr_num(top_text[encr1], alphabet)  # Y*
                    
                    lang2 = get_bigr_num(top_lang[clear2], alphabet)# X**
                    text2 = get_bigr_num(top_text[encr2], alphabet) # Y**
                    
                    #(lang1 - lang2) * a = (text1 - text2) mod m^2
                    # A*x = B mod m^2
                    #B = text1 - A*lang1 mod m^2
                    
                    m = len(alphabet)
                    A = lang1 - lang2
                    B = text1 - text2
                    
                    results = solve_equation(A, B, pow(m, 2))
                    if type(results) == list:
                        for i in range(0, len(results)):
                            _A = results[i]
                            _B = (text1 - _A * lang1) % pow(m, 2)
                            decrypted = decrypt_text(_A, _B, text, alphabet)
                            detect_lang(_A, _B, decrypted, alphabet)